# Generate index weights using dedicated service - C#

### Overview
Reference data encompasses a wide range of specification data about:
* financial instrument such as asset class, symbol, maturity, etc.
* counterparties such as issuer information.
* pricing such as Open, high low and close prices.

The challenge with reference data is that it tends to be sourced from multiple sources: internal, counterparties and providers.

Reference data dedicated service is a result of various data sources and fields sourcing, validation, cross-checking and normalization.<br>

This sample requests **on-demand** exchange data and enables fields mapping by calling a dedicated static data service, making available clean data.

### Inputs/outputs
Index weights sample is designed to help you searching and requesting index components reference data by using a wide range of commonly used fields.

It takes an index code as per input filter and returns index components and weights.

### Services used
This sample uses *gRPC requests* in order to retrieve index components and weights from the hosted service. The queried endpoint in this script are:
* *ComponentsService*: to directly retrieve index data objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Indices.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*

***

# Run Index weights sample

### Step 1: Install packages

In [1]:
#i "nuget: file://~/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: CsvHelper"
#r "nuget: XPlot.Plotly.Interactive"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources file://~/.nuget/packages/ https://api.nuget.org/v3/index.json Installed Packages CsvHelper, 29.0.0 Systemathics.Apis, 0.13.2-pre XPlot.Plotly.Interactive, 4.0.6

Loading extensions from `XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [2]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Indices.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
// Import/export csv files
using CsvHelper;
using System.IO;
using System.Globalization;
// Plot graphs
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [3]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Request creation

The following code snippet enables to select the **index** by its *Name*:

In [4]:
// Set index name
var index = "nasdaq 100";

The following code snippets call the service, generate the request and return the reply: 

In [5]:
// Instantiate the indice components service
var service = new ComponentsService.ComponentsServiceClient(channel);

// Generate the indice components request
var request = new ComponentsRequest { Identifier = index };

// Process the indice components response
var reply = service.Components(request, headers);

// Display the results
display(reply.Components)

index Identifier Weight 0 Exchange Ticker Provider XNGS WBA ICE 0.286 1 Exchange Ticker Provider XNGS AAPL ICE 13.752 2 Exchange Ticker Provider XNGS PCAR ICE 0.302 3 Exchange Ticker Provider XNGS ANSS ICE 0.18 4 Exchange Ticker Provider XNGS AEP ICE 0.424 5 Exchange Ticker Provider XNGS WDAY ICE 0.29 6 Exchange Ticker Provider XNGS ROST ICE 0.287 7 Exchange Ticker Provider XNGS MELI ICE 0.41 8 Exchange Ticker Provider XNGS LCID ICE 0.215 9 Exchange Ticker Provider XNGS CPRT ICE 0.251 10 Exchange Ticker Provider XNGS MCHP ICE 0.329 11 Exchange Ticker Provider XXXX CEG ICE 0.275 12 Exchange Ticker Provider XNGS MTCH ICE 0.117 13 Exchange Ticker Provider XNGS ADP ICE 0.932 14 Exchange Ticker Provider XNGS EA ICE 0.335 15 Exchange Ticker Provider XNGS KHC ICE 0.416 16 Exchange Ticker Provider XNGS ADI ICE 0.727 17 Exchange Ticker Provider XNGS PAYX ICE 0.389 18 Exchange Ticker Provider XNGS NTES ICE 0.098 19 Exchange Ticker Provider XNGS BIDU ICE 0.16 (82 more)

### Step 4: Retrieve index data

#### 4.1 Export index data

The following code snippet exports index data to a *csv file*:

In [6]:
var d = new DirectoryInfo("output"); if (!d.Exists) { d.Create(); }
using (var writer = new StreamWriter($"output/{index.ToLowerInvariant()} weights.csv"))
using (var csv = new CsvWriter(writer, CultureInfo.InvariantCulture))
{
    csv.WriteRecords(reply.Components);
}

#### 4.2 Visualize index data

The following code snippet displays index components and weights. <br>*`XPlot.Plotly`* is used as per open source display package. 

In [7]:
var data = reply.Components.Select(elt => new Tuple<String, double>(elt.Identifier.Ticker, elt.Weight));

In [8]:
var chart = Chart.Pie(data);
chart.WithTitle($"{index} components and weights");
chart.Width = 1500;
chart.Height = 800;
display(chart);

<!DOCTYPE html>